In [36]:
import pickle
import pandas as pd
import copy
import os
import numpy as np
# import torch
from omegaconf import OmegaConf
import ast
import sys
# from definitions import RESULTS_DIR, ROOT_DIR_INSIDE
import pickle
import pandas as pd
import copy
import os
import numpy as np
import ast
# from xgboost import XGBClassifier
from sklearn import metrics
# root_dir = ROOT_DIR_INSIDE
# config = OmegaConf.load(f"{sys.argv[1]}/config.yaml")
input_data_dir = '/Users/projects_data/project_MLPhenotypesMRIGWAS/ml_generated_data/23_03_30/predictiveModel/pd_imaging.h5ad'
fname = os.path.basename(input_data_dir).split('.')[0]
# data_dir = f"{RESULTS_DIR}/prediction_results/example"
all_predictions = {}

def get_obs_data_anndata(fold_name):
    import anndata as ad
    adata = ad.read("/Users/projects_data/project_MLPhenotypesMRIGWAS/ml_generated_data/23_03_30/predictiveModel/pd_imaging.h5ad")
    split_column = "subject_id-DIAGNOSIS-StratifiedGroupKFold_GroupShuffleSplit_42_42"
    train_data = adata[adata.obsm['train_val_test_split'][~(adata.obsm['train_val_test_split'][split_column] .isin ([fold_name]))].index]
    val_data = adata[adata.obsm['train_val_test_split'][adata.obsm['train_val_test_split'][split_column] .isin ([fold_name])].index]
    X_train = np.array(train_data.X, dtype=np.float32)
    X_val = np.array(val_data.X, dtype=np.float32)
    y_train = np.array(train_data.obs['DIAGNOSIS'], dtype=np.float32)
    y_val = np.array(val_data.obs['DIAGNOSIS'], dtype=np.float32)
    return val_data.obs

obs = pd.concat([get_obs_data_anndata('fold1'), get_obs_data_anndata('fold2'), get_obs_data_anndata('fold3')])

In [37]:
from pathlib import Path
RESULTS_DIR = Path("/Users/EssentialCodeBase/project_MLPhenotypesMRIGWAS/block1_BaselineClassification/training/23_03_30/results_data")

In [38]:
for model_name in ["LogisticRegression", "skorchMLPClassifier", "XGBClassifier"]:
    with open(RESULTS_DIR / f'predictiveAnalysis/execute_ensemble_models/ensemble_data/{fname}_{model_name}_best_model_prediction.pkl', 'rb') as t:
        all_predictions.update({f"{fname}#{i}":j for i,j in pickle.load(t).items()})

In [39]:
csv_file_location = "/Users/EssentialCodeBase/project_MLPhenotypesMRIGWAS/block1_BaselineClassification/training/23_03_30/results_data/prediction_results/example/apr5/IMAGING_PDPREDICTION_CLASSIFIER/basic_numerical$imaging__pddiagnosisprediction_train100$feature$sample"
for model_name in ['LogisticRegression', 'skorchMLPClassifier', 'XGBClassifier']:
    leaderboard_df = pd.read_csv(f"{csv_file_location}/csv_files/leaderboard_df.csv")
    agg_leaderboard_df = pd.read_csv(f"{csv_file_location}/csv_files/agg_leaderboard_df.csv")
    leaderboard_df = leaderboard_df[leaderboard_df['model_name'].isin([model_name])]
    agg_leaderboard_df = agg_leaderboard_df[agg_leaderboard_df['model_name'].isin([model_name])]
    config_mapping = dict(zip(leaderboard_df['config_index'], leaderboard_df['config'].map(lambda x: ast.literal_eval(x))))
    config_index1 = agg_leaderboard_df.iloc[0]['config_index']
    main_config_grid = OmegaConf.load(f"/Users/EssentialCodeBase/BaselineClassification/PredictiveModeling/src_basic/optuna_hyperparameters_with_index.json")
    classifier_params = main_config_grid['IMC'][model_name]['hp0']
    classifier_params.update(config_mapping[config_index1])
    dfg = classifier_params
    for key, val in dfg.items():
        try:
            dfg[key] = round(val, 2)
        except:
            pass
    print (dfg)

{'type': 'optuna', 'num_trials': 20, 'multi_class': 'multinomial', 'random_state': 42, 'FeatureSelection': 'IDENTITY', 'solver': 'saga', 'max_iter': 100, 'C': 184.46, 'penalty': 'l2'}
{'type': 'optuna', 'num_trials': 100, 'hidden_layer_sizes': [64, 32], 'max_epochs': 500, 'FeatureSelection': 'LASSO', 'optimizer': 'adamw', 'early_stopping': 1, 'random_state': 42, 'layers': 2, 'layer_size': [1, 512], 'learning_rate': [1e-05, 0.01], 'dropout': [0.0, 0.5], 'weight_decay': [1e-06, 0.001], 'layer_size0': 85, 'layer_size1': 337, 'module__dropout': 0.46, 'lr': 0.0, 'optimizer__weight_decay': 0.0}
{'type': 'optuna', 'num_trials': 200, 'objective': 'multi:softprob', 'seed': 42, 'nthread': 4, 'silent': 1, 'n_estimators': 45, 'booster': 'gbtree', 'learning_rate': 0.0, 'reg_lambda': 7.55, 'gamma': 40.45, 'reg_alpha': 0.0, 'max_depth': 10, 'min_child_weight': 0.0, 'subsample': 0.52, 'colsample_bytree': 0.69, 'tree_method': 'gpu_hist', 'predictor': 'cpu_predictor', 'max_bin': 289, 'FeatureSelection':

In [20]:
model_list = list(all_predictions)
model_list

from collections import defaultdict
model_list = list(all_predictions)
ll_loss_df = defaultdict(list)
full_dataframe_list = []
for e, model_name in enumerate(model_list):
    data_list = []
    for e, fold_name in enumerate(sorted(list(all_predictions[model_name]))):
        logloss_value = metrics.log_loss(all_predictions[model_name][fold_name]['y_val'], all_predictions[model_name][fold_name]['y_pred_val'])
        auc_value = metrics.roc_auc_score(all_predictions[model_name][fold_name]['y_val'], all_predictions[model_name][fold_name]['y_pred_val'][:, 1] , multi_class='ovo' )
        ll_loss_df['fold'].append(fold_name)
        ll_loss_df['model_name'].append(model_name)
        ll_loss_df['ll_loss'].append(logloss_value)
        ll_loss_df['micro_auc'].append(auc_value)
        ll_loss_df['accuracy'].append(  metrics.accuracy_score(all_predictions[model_name][fold_name]['y_val'], np.argmax(all_predictions[model_name][fold_name]['y_pred_val'], axis=1 ) )   )
        macro_auc = np.mean([metrics.roc_auc_score(all_predictions[model_name][fold_name]['y_val'] == i, all_predictions[model_name][fold_name]['y_pred_val'][:,i]) for i in range(2)])
        ll_loss_df['macro_auc'].append(macro_auc)
        temp1 = pd.DataFrame(all_predictions[model_name][fold_name]['y_val'], columns=[f'y_true'])
        temp1['fold'] = fold_name
        temp2 = pd.DataFrame(all_predictions[model_name][fold_name]['y_pred_val'], columns=[f'{model_name}-y_pred{i}' for i in range(all_predictions[model_name][fold_name]['y_pred_val'].shape[1])])
        data_list.append(pd.concat([temp1, temp2], axis=1).set_index(['fold', 'y_true']))
    full_dataframe_list.append(pd.concat(data_list, axis=0))
    

In [21]:
def f_mean(x):
    return str(round(np.mean(list(x)), 4)) + '+/-' + str(np.std(list(x)).round(2))
    
full_dataframe = pd.concat(full_dataframe_list, axis=1)
print ('-'*100)
print (pd.DataFrame(ll_loss_df).groupby('model_name').agg(f_mean))


L = full_dataframe.copy()
L.index = obs.index
# L.reset_index().to_csv("embeddings/all_concat_full_dataframe.csv", index=False)

def get_training_data_numpy(fold_name):
    full_dataframe_folds = full_dataframe.reset_index()
    full_dataframe_val = full_dataframe_folds[full_dataframe_folds['fold']==fold_name]
    full_dataframe_train = full_dataframe_folds[~(full_dataframe_folds['fold']==fold_name)]
    X_train = full_dataframe_train.iloc[:, 2:].values.astype(np.float32)
    X_val = full_dataframe_val.iloc[:, 2:].values.astype(np.float32)
    y_train = full_dataframe_train.iloc[:, 1].values.astype(np.int64)
    y_val = full_dataframe_val.iloc[:, 1].values.astype(np.int64)
    return X_train, X_val, y_train, y_val


----------------------------------------------------------------------------------------------------
                                      ll_loss      micro_auc       accuracy  \
model_name                                                                    
pd_imaging#LogisticRegression   0.6381+/-0.04  0.6474+/-0.03  0.7159+/-0.02   
pd_imaging#XGBClassifier         0.6846+/-0.0      0.5+/-0.0  0.7467+/-0.04   
pd_imaging#skorchMLPClassifier  0.5666+/-0.05  0.5729+/-0.03  0.7467+/-0.04   

                                    macro_auc  
model_name                                     
pd_imaging#LogisticRegression   0.6474+/-0.03  
pd_imaging#XGBClassifier            0.5+/-0.0  
pd_imaging#skorchMLPClassifier  0.5729+/-0.03  


/var/folders/y5/tr1yp82s2ns2_4g_6flxk45jrqb07t/T/ipykernel_78953/317826522.py:6: FutureWarning: ['fold'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  print (pd.DataFrame(ll_loss_df).groupby('model_name').agg(f_mean))


In [22]:

from omegaconf import OmegaConf
from sklearn.linear_model import LogisticRegression
# from lightgbm import LGBMClassifier
# from PredictiveModeling.src_basic.skorch_models import skorchMLPClassifier

# optuna_config_grid = OmegaConf.load(f"{ROOT_DIR_INSIDE}/PredictiveModeling/src_basic/{config['tuning_type']}_hyperparameters_with_index.json")

L = full_dataframe.copy()
L.index = obs.index

In [23]:
import pickle
with open(RESULTS_DIR / f'predictiveAnalysis/execute_ensemble_models/ensemble_data/{fname}_ensembleWeights_best_model_prediction.pkl', 'rb') as t:
    ensemble_weights = pickle.load(t)

In [24]:
ensemble_weights

{'pd_imaging#LogisticRegression': 0.42749628067654155,
 'pd_imaging#skorchMLPClassifier': 0.550098560653293,
 'pd_imaging#XGBClassifier': 0.022405158670165418}

In [25]:
d_ensemble = {}
for col in full_dataframe.columns:
    w = ensemble_weights[col.split('-')[0]]
    pred_name = col.split('-')[1]
    d_ensemble['ensemble-'+pred_name] = d_ensemble.get('ensemble-'+pred_name, 0) + w * full_dataframe[col].values
    

ensembled_prediction = pd.DataFrame(d_ensemble)
ensembled_prediction.index = full_dataframe.index
ensembled_prediction = ensembled_prediction.reset_index()
from collections import defaultdict
def f_mean(x):
    return str(round(np.mean(list(x)), 4)) + '+/-' + str(np.std(list(x)).round(2))
    
g = list(ensembled_prediction.groupby('fold'))
cv_all_results = defaultdict(list)

for i in range(len(g)):  
    temp = g[i][1]
    cv_all_results['const'].append(0)
    cv_all_results['fold'].append(g[i][0])
    macro_auc = np.mean([metrics.roc_auc_score(temp['y_true'].values == i, temp.iloc[:, 2:].values[:,i]) for i in range(2)])
    cv_all_results['cv_macro_auc'].append(macro_auc)
    cv_all_results['cv_logloss'].append(  metrics.log_loss(temp['y_true'].values, temp.iloc[:, 2:])   )
    cv_all_results['cv_micro_auc'].append(  metrics.roc_auc_score(temp['y_true'].values, temp.iloc[:, 3] , multi_class='ovo' ) )
    cv_all_results['cv_accuracy'].append(  metrics.accuracy_score(temp['y_true'].values, np.argmax(temp.iloc[:, 2:].values, axis=1 ) )   )
    

avg = pd.DataFrame(cv_all_results).groupby('const').agg(f_mean).reset_index()
avg['fold'] = 'fold'


cv_all_results_df = pd.DataFrame(cv_all_results)
print ('-'*100)
print (pd.concat([cv_all_results_df, avg]))


print ('-'*100)
print (pd.DataFrame(ll_loss_df).groupby('model_name').agg(f_mean))


----------------------------------------------------------------------------------------------------
   const   fold   cv_macro_auc     cv_logloss   cv_micro_auc    cv_accuracy
0      0  fold1       0.642857       0.515988       0.642857       0.758621
1      0  fold2       0.611768        0.53344       0.611768       0.758621
2      0  fold3        0.67538       0.591843        0.67538       0.680782
0      0   fold  0.6433+/-0.03  0.5471+/-0.03  0.6433+/-0.03  0.7327+/-0.04
----------------------------------------------------------------------------------------------------
                                      ll_loss      micro_auc       accuracy  \
model_name                                                                    
pd_imaging#LogisticRegression   0.6381+/-0.04  0.6474+/-0.03  0.7159+/-0.02   
pd_imaging#XGBClassifier         0.6846+/-0.0      0.5+/-0.0  0.7467+/-0.04   
pd_imaging#skorchMLPClassifier  0.5666+/-0.05  0.5729+/-0.03  0.7467+/-0.04   

                      

/var/folders/y5/tr1yp82s2ns2_4g_6flxk45jrqb07t/T/ipykernel_78953/4155623755.py:29: FutureWarning: ['fold'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  avg = pd.DataFrame(cv_all_results).groupby('const').agg(f_mean).reset_index()
/var/folders/y5/tr1yp82s2ns2_4g_6flxk45jrqb07t/T/ipykernel_78953/4155623755.py:39: FutureWarning: ['fold'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  print (pd.DataFrame(ll_loss_df).groupby('model_name').agg(f_mean))


In [26]:
len(cv_all_results)

6

In [27]:
cv_all_results

defaultdict(list,
            {'const': [0, 0, 0],
             'fold': ['fold1', 'fold2', 'fold3'],
             'cv_macro_auc': [0.6428571428571429,
              0.611768115942029,
              0.6753804614629357],
             'cv_logloss': [0.5159881400089227,
              0.533439608317454,
              0.5918430118146706],
             'cv_micro_auc': [0.6428571428571429,
              0.611768115942029,
              0.6753804614629356],
             'cv_accuracy': [0.7586206896551724,
              0.7586206896551724,
              0.6807817589576547]})

In [28]:
ensembled_prediction

,fold,y_true,ensemble-y_pred0,ensemble-y_pred1
0,fold1,0.0,0.513898,0.486102
1,fold1,1.0,0.236703,0.763297
2,fold1,0.0,0.411106,0.588894
3,fold1,0.0,0.226822,0.773178
4,fold1,1.0,0.231685,0.768315
...,...,...,...,...
911,fold3,1.0,0.152408,0.847592
912,fold3,1.0,0.219231,0.780769
913,fold3,1.0,0.213356,0.786644
914,fold3,1.0,0.203344,0.796656


In [33]:
ensembled_prediction.to_csv("data/pd_ensembled_prediction.csv", index=False)

In [34]:
print ("Number of subject", obs[obs['PHENO']=='Control'].index.map(lambda x: '-'.join(x.split('-')[:2])))
print ("Number of unique subject", obs[obs['PHENO']=='Control'].index.map(lambda x: '-'.join(x.split('-')[:2])).unique())

Number of subject Index(['PPMI-4067', 'PPMI-3055', 'PPMI-3064', 'PPMI-3479', 'PPMI-3075',
       'PPMI-3855', 'PPMI-3521', 'PPMI-3809', 'PPMI-3901', 'PPMI-3855',
       ...
       'PPMI-3389', 'PPMI-3351', 'PPMI-3106', 'PPMI-3106', 'PPMI-3411',
       'PPMI-3852', 'PPMI-3316', 'PPMI-3112', 'PPMI-3389', 'PPMI-3405'],
      dtype='object', name='ID', length=232)
Number of unique subject Index(['PPMI-4067', 'PPMI-3055', 'PPMI-3064', 'PPMI-3479', 'PPMI-3075',
       'PPMI-3855', 'PPMI-3521', 'PPMI-3809', 'PPMI-3901', 'PPMI-4004',
       ...
       'PPMI-3779', 'PPMI-3600', 'PPMI-3112', 'PPMI-3316', 'PPMI-3635',
       'PPMI-3106', 'PPMI-3389', 'PPMI-3351', 'PPMI-3411', 'PPMI-3405'],
      dtype='object', name='ID', length=145)


In [35]:
print ("Number of subject", obs[obs['PHENO']=='PD'].index.map(lambda x: '-'.join(x.split('-')[:2])))
print ("Number of unique subject", obs[obs['PHENO']=='PD'].index.map(lambda x: '-'.join(x.split('-')[:2])).unique())

Number of subject Index(['PPMI-3789', 'PPMI-3789', 'PPMI-3482', 'PPMI-3089', 'PPMI-3789',
       'PPMI-3473', 'PPMI-3051', 'PPMI-3077', 'PPMI-3661', 'PPMI-3789',
       ...
       'PPMI-4038', 'PPMI-3126', 'PPMI-3130', 'PPMI-4038', 'PPMI-3126',
       'PPMI-4038', 'PPMI-3028', 'PPMI-3609', 'PPMI-4051', 'PPMI-4122'],
      dtype='object', name='ID', length=684)
Number of unique subject Index(['PPMI-3789', 'PPMI-3482', 'PPMI-3089', 'PPMI-3473', 'PPMI-3051',
       'PPMI-3077', 'PPMI-3661', 'PPMI-3764', 'PPMI-3470', 'PPMI-4055',
       ...
       'PPMI-3802', 'PPMI-3060', 'PPMI-3415', 'PPMI-4091', 'PPMI-3126',
       'PPMI-4038', 'PPMI-3028', 'PPMI-3609', 'PPMI-4051', 'PPMI-4122'],
      dtype='object', name='ID', length=319)


In [32]:
obs

,PHENO,DIAGNOSIS,subject_id,SID,TASK
ID,,,,,
PPMI-4067-20121215-000,Control,0,PPMI-4067,PPMI-4067-20121215-000,TASK1
PPMI-3789-20161115-001,PD,1,PPMI-3789,PPMI-3789-20161115-001,TASK1
PPMI-3055-20101215-000,Control,0,PPMI-3055,PPMI-3055-20101215-000,TASK1
PPMI-3064-20110515-000,Control,0,PPMI-3064,PPMI-3064-20110515-000,TASK1
PPMI-3789-20141115-000,PD,1,PPMI-3789,PPMI-3789-20141115-000,TASK1
...,...,...,...,...,...
PPMI-4038-20130415-000,PD,1,PPMI-4038,PPMI-4038-20130415-000,TASK1
PPMI-3028-20130315-000,PD,1,PPMI-3028,PPMI-3028-20130315-000,TASK1
PPMI-3609-20110515-000,PD,1,PPMI-3609,PPMI-3609-20110515-000,TASK1
